In [1]:
using Revise, ReachabilityAnalysis, BenchmarkTools
const RA = ReachabilityAnalysis
using Plots

In [2]:
include("/home/mforets/.julia/dev/ReachabilityAnalysis/test/models/hybrid/embrake.jl")

embrake_pv_2 (generic function with 1 method)

## No prameter variation

No jitter:

In [ ]:
prob = embrake_no_pv(ζ=0.0, Tsample=1e-4);

In [ ]:
@time sol = solve(prob, alg=GLGM06(δ=1e-8, max_order=3), max_jumps=1000, static=true);

In [ ]:
diameter(set(project(sol[end][end], vars=(1)))) # variable I

In [ ]:
length(sol) * length(sol[1])

In [ ]:
diameter(set(project(sol[end][end], vars=(2)))) # variable x

In [ ]:
@time sol = solve(prob, alg=GLGM06(δ=1e-8, max_order=10), max_jumps=200, static=false);

In [ ]:
median([@elapsed sol = solve(prob, alg=GLGM06(δ=1e-8, max_order=3), max_jumps=1000, static=true) for _ in 1:10])
# length(sol) * length(sol[1])

In [ ]:
median([@elapsed sol = solve(prob, alg=BOX(δ=1e-8), max_jumps=100, static=true) for _ in 1:10])

With jitter:

In [ ]:
prob = embrake_no_pv(ζ=1e-7, Tsample=1e-4);

#median([@elapsed sol = solve(prob, alg=GLGM06(δ=1e-8, max_order=3), max_jumps=1000, static=true) for _ in 1:10])

#length(sol) * length(sol[1])

In [ ]:
@time sol = solve(prob, alg=GLGM06(δ=1e-8, max_order=3), max_jumps=1000, static=true)

@show diameter(set(project(sol[end][end], vars=(1)))) # variable I
@show diameter(set(project(sol[end][end], vars=(2)))) # variable x

In [ ]:
prob = embrake_no_pv(ζ=1e-7, Tsample=1e-4);
sol_pv = solve(prob, alg=GLGM06(δ=1e-8), max_jumps=3, static=true);
tspan(sol_pv[end][end])
length(sol_pv) * length(sol_pv[4])

In [ ]:
prob = embrake_no_pv(ζ=0.0, Tsample=1e-4);
sol = solve(prob, alg=GLGM06(δ=1e-8), max_jumps=3, static=true)
tspan(sol[end][end])
length(sol) * length(sol[1])

## With parameter variation (pv1)

In [3]:
prob = embrake_pv_1(ζ=0.0, Tsample=1e-4);

In [4]:
using UnPack
using ReachabilityAnalysis: AbstractContinuousPost, step_size

function post(alg::AbstractContinuousPost, ivp::IVP{<:HACLD1}, max_jumps)

    X0 = initial_state(ivp)
    ha = system(ivp)
    @unpack sys, rmap, Tsample, ζ = ha
    t0 = 0.0
    δ = step_size(alg)

    # solve first interval
    prob = IVP(sys, X0)
    αlow = (Tsample - ζ)/δ
    NLOW = ceil(Int, αlow)
    if NLOW == 0
        error("inconsistent choice of parameters: (Tsample - ζ)/δ = $αlow " *
              "but it should be positive")
    end
    αhigh = (Tsample + ζ)/δ
    NHIGH = ceil(Int, αhigh)
    return prob, NHIGH, alg
end

#=
    # preallocate output vector of flowpipes
    # TODO: use a custom flowpipe type, eg. HybridFlowpipe
    FT = typeof(flowpipe(sol))
    out = Vector{FT}()
    sizehint!(out, max_jumps+1)

    @inbounds for k in 1:max_jumps+1

        # add time interval
        aux = sol[1:NLOW] # Vector(sol(0 .. Tsample-ζ))

        push!(out, shift(Flowpipe(aux), t0))

        t0 += tstart(aux[end])

        # Xend = sol(Tsample-ζ .. Tsample+ζ) |> Vector |> Flowpipe |> Convexify |> set
        Xend = sol[NLOW:NHIGH] |> Vector |> Flowpipe |> Convexify |> set

        prob = IVP(sys, rmap(Xend))
        sol = solve(prob, NSTEPS=NHIGH, alg=alg; kwargs...)
    end

    return HybridFlowpipe(out)
end
=#

post (generic function with 1 method)

In [131]:
prob_cont, NHIGH, alg = post(ASB07(δ=1e-5, max_order=1), prob, 0);

In [132]:
alg.δ

1.0e-5

In [134]:
@time sol = solve(prob_cont, NSTEPS=3_000, alg=alg, static=false);

  1.887836 seconds (18.12 M allocations: 2.829 GiB, 14.54% gc time)


|NSTEPS| Runime|
|------|------|
|1000 | 0.74|
|2000 | 1.67|
|3000 | 7.15|

In [135]:
0.1 / tend(sol)

3.333333333332708

6.66

In [137]:
6666 / 60. / 60.

1.8516666666666666

In [90]:
@time sol = solve(prob_cont, NSTEPS=100, alg=alg, static=true); # 

Zonotope{Float64,SArray{Tuple{4},Float64,1,4},SArray{Tuple{4,29},Float64,2,116}}


MethodError: MethodError: Cannot `convert` an object of type 
  Zonotope{Float64{},SArray{Tuple{4},Float64{},1,4},SArray{Tuple{4,41},Float64{},2,164}} to an object of type 
  Zonotope{Float64{},SArray{Tuple{4},Float64{},1,4},SArray{Tuple{4,29},Float64{},2,116}}
Closest candidates are:
  convert(::Type{#s16} where #s16<:Union{Number, T}, !Matched::MultivariatePolynomials.AbstractPolynomialLike{T}) where T at /home/mforets/.julia/packages/MultivariatePolynomials/Mpv8c/src/conversion.jl:16
  convert(::Type{T}, !Matched::T) where T at essentials.jl:171
  Zonotope{Float64,SArray{Tuple{4},Float64,1,4},SArray{Tuple{4,29},Float64,2,116}}(::Any, !Matched::Any) where {N<:Real, VN<:AbstractArray{N,1}, MN<:AbstractArray{N,2}} at /home/mforets/.julia/dev/LazySets/src/Sets/Zonotope.jl:100

In [ ]:
# sol = solve(prob, alg=ASB07(δ=1e-8, max_order=3), max_jumps=0, sta);

In [5]:
sol = solve(prob, NSTEPS=NHIGH, alg=alg; kwargs...)

UndefVarError: UndefVarError: NHIGH not defined

In [173]:
prob = embrake_pv_1(ζ=0.0, Tsample=1e-4);
@time sol = solve(prob, alg=ASB07(δ=1e-7, max_order=1), max_jumps=100);
#length(sol) * length(sol[1])
tspan(sol)

 45.298571 seconds (208.59 M allocations: 32.478 GiB, 50.41% gc time)


[0, 0.0101001]

In [174]:
@show diameter(set(project(sol[end][end], vars=(1)))) # variable I
@show diameter(set(project(sol[end][end], vars=(2)))) # variable x

diameter(set(project((sol[end])[end], vars = 1))) = 11.872188657505149
diameter(set(project((sol[end])[end], vars = 2))) = 0.0001469325130479364


0.0001469325130479364

In [171]:
LazySets.radius(set(sol.F[end][end]))

7.4940023993559635

In [ ]:
prob = embrake_pv_1(ζ=1e-7, Tsample=1e-4)
# TODO: force_static?

@time sol = solve(prob, alg=ASB07(δ=1e-8, max_order=3), max_jumps=2);
length(sol) * length(sol[1])

## With parameter variation (pv2)

In [ ]:
prob = embrake_pv_2(ζ=0.0, Tsample=1e-4)
# TODO: force_static

@time sol = solve(prob, alg=ASB07(δ=1e-8), max_jumps=1000);
length(sol) * length(sol[1])

In [ ]:
prob = embrake_pv_2(ζ=1e-7, Tsample=1e-4)
# TODO: force_static

@time sol = solve(prob, alg=ASB07(δ=1e-8), max_jumps=1000);
length(sol) * length(sol[1])

## Requirements

1) Variable $x(t)$ is such that $\Vert x(t) - x_0 \Vert \leq \varepsilon$ for all $t \geq t_1$. We take $t_1 = 23ms$ and $x_0 = 0.05$.

In [ ]:
x0 = 0.05
ε = 2e-3

2) The speed does is not bigger than a certain threshold  value once the disc makes contact: $\vert v(t) \vert \leq v_{max}$ for all $t \geq t_1$. We take $t_1 = 23ms$ and $v_{max} = 2mm/s$.

### Requirement 1

In [ ]:
function requirement_1(sol::HybridFlowpipe{N}) where {N}

    Dx⁺ = [0, 1.0, 0, 0]
    Dx⁻ = [0, -1.0, 0, 0]
    dirs = [Dx⁺, Dx⁻]

    vtempl = Vector{TemplateReachSet{N, Vector{N}}}()
    sizehint!(vtempl, length(sol) * length(sol[1]))
    
    for fp in sol
        for X in fp
            Rtemplate = overapproximate(X, dirs)
            push!(vtempl, Rtemplate)
        end
    end
    return Flowpipe(vtempl)
end

In [ ]:
fp = flowpipe(sol);
@time r1 = requirement_1(fp);

In [ ]:
maximos = [sup_func(X, 1) for X in r1];

In [ ]:
length(maximos)

In [ ]:
using Plots

In [ ]:
maximos[end]

In [ ]:
plot(maximos[1:1_000:end], seriestype = :scatter, lab="")

$|x(t) - x_0|$ vale $x(t) - x_0$ si $x(t) \geq x_0$ o $x_0 - x(t)$ si $x(t) < x_0$.

In [ ]:
tend(r1)

In [ ]:
const IA = IntervalArithmetic

t1 = 98e-3 #23e-3
tf = tend(r1)
y = r1(IA.Interval(t1, tf))

max_D⁺ = maximum(sup_func(X, 1) for X in y)
@show max_D⁺ 
@show max_D⁺ < x0 + ε

max_D⁻ = maximum(sup_func(X, 2) for X in y)
@show max_D⁻
@show max_D⁻ < -x0 + ε

In [ ]:
-x0 + ε

In [ ]:
length(sol[1])

In [ ]:
plot([project(X, [0, 2]) for X in sol[600][1:100:end]], vars=(0, 2))
plot!(x -> x0 + ε, color=:red)
plot!(x -> x0 - ε, color=:red)

In [ ]:
tspan(sol[900])

In [ ]:
x0 - maximum(sup_func(X, 2) for X in y)

In [ ]:
[Interval(tspan(X)) × Interval(sup_func(X, 1), sup_func(X, 1)) for X in r1];

In [ ]:
using Plots

In [ ]:
plot([Interval(tspan(X)) × Singleton([sup_func(X, 1)]) for X in r1])

In [ ]:
plot!([Interval(tspan(X)) × Singleton([-sup_func(X, 2)]) for X in r1])

In [ ]:
# project(sol[1], [2]) # hacer
# project(sol[1][1], (2)) # HACER

intervalo_test = Interval(x0 - ε, x0 + ε)

X = [[set(project(X, [2])) for X in sol_i] for sol_i in sol];

In [ ]:
X[2][10] ⊆ intervalo_test

### Requirement 2

In [ ]:
# assumes that v = K/(i*drot) * I and
# I is the first (1) variable in the flowpipe
function velocity(sol, K, i, drot)
    M = hcat(K/(i*drot))
    if isa(M, Interval)
        M = IntervalMatrix(M)
    end

    V_fp = Vector{SparseReachSet}()
    for fp in sol
        for X in fp
            I_Δt = overapproximate(M * set(project(X, [1])), Zonotope)
            Δt = tspan(X)
            push!(V_fp, SparseReachSet(I_Δt, Δt, [1]))
        end
    end

    return V_fp
end

function requirement_2(sol::Vector{<:Flowpipe{N}}) where {N}

    Dx⁺ = [0, 1.0, 0, 0]
    Dx⁻ = [0, -1.0, 0, 0]
    dirs = [Dx⁺, Dx⁻]

    vtempl = Vector{TemplateReachSet{N, Vector{N}}}()
    sizehint!(vtempl, length(sol) * length(sol[1]))
    
    for fp in sol
        for X in fp
            sfunc = [ρ(Dx⁺, X), ρ(Dx⁻, X)]
            Δt = tspan(X)
            R = TemplateReachSet(dirs, sfunc, Δt)
            push!(vtempl, R)
        end
    end
    return vtempl
end